
## Get US regional banks industry data

In [ ]:
import numpy as np 
import pandas as pd

import requests

import yfinance as yf

In [ ]:
fmp_apikey = 'insert fmp api key'

In [ ]:
try:
    # For Python 3.0 and later
    from urllib.request import urlopen
except ImportError:
    # Fall back to Python 2's urllib2
    from urllib2 import urlopen

import json

def get_jsonparsed_data(url):
    """
    Receive the content of ``url``, parse it as JSON and return the object.

    Parameters
    ----------
    url : str

    Returns
    -------
    dict
    """
    response = urlopen(url)
    data = response.read().decode('utf-8')
    return json.loads(data)


In [ ]:
banks_us_url = "https://financialmodelingprep.com/api/v3/stock-screener?industry=Banks_Regional&exchange=NASDAQ,NYSE,AMEX&apikey={}".format(fmp_apikey)
# US Banks (Regional)

In [ ]:
banks_us = get_jsonparsed_data(banks_us_url)

In [ ]:
pd.set_option('display.max_rows', len(banks_us))

In [ ]:
banks_us_df = pd.DataFrame(banks_us)

In [ ]:
banks_us_df.sort_values('symbol')

In [ ]:
raw_company_list = banks_us_df['symbol'].values


Filter out "non-primary" shares 

In [ ]:
tick_5_letters = []

for tick in banks_us_df['symbol'].items():
    if len(tick[1]) == 5:
        tick_5_letters.append(tick[1])   

In [ ]:
tick_5letters_df = banks_us_df[banks_us_df['symbol'].apply(lambda tick: (len(tick) == 5) & (tick[-1].upper() != 'A'))]
tick_5letters = tick_5letters_df.symbol.values

In [ ]:
tick_5letters

In [ ]:
depositary_receipts_df = banks_us_df[banks_us_df['companyName'].apply(lambda name: ('ADR' in name) or
                                                                      ('ADS' in name) or
                                                                      ('Depositary' in name) or
                                                                      ('depositary' in name) or
                                                                      ('De' in name.split(' ')) or
                                                                      ('de' in name.split(' ')))]

In [ ]:
depositary_receipts_df 

In [ ]:
tick_dep_rec_no5 = depositary_receipts_df[depositary_receipts_df['symbol'].apply(lambda tick: len(tick) < 5)].symbol.values

In [ ]:
# check other ADRs
industry_with_adr = pd.read_csv('List of Stocks For An Industry.csv') #data with ADR
industry_with_adr_df = pd.DataFrame(industry_with_adr)

In [ ]:
adrs = industry_with_adr_df[industry_with_adr_df['Category2'] == 'ADRs']

In [ ]:
adrs_tick = adrs['Symbol'].values

In [ ]:
new_adrs_list = list(set(adrs_tick).difference(depositary_receipts_df['symbol'].values))

In [ ]:
#adrs_tick_ext_df = banks_us_df[banks_us_df['symbol'].apply(lambda x: (x=='AVAL') or (x=='BCH') or (x=='CIB') or (x=='GGAL') or (x=='HDB') or (x=='IBN') or (x=='ITCB') or (x=='KB') or (x=='LYG'))]

In [ ]:
adrs_tick_ext_df = banks_us_df[banks_us_df['symbol'].isin(new_adrs_list)]

In [ ]:
adrs_tick_ext = adrs_tick_ext_df['symbol'].values

In [ ]:
other_adr = ['BBAR']
other_curr = ['DB', 'IFS']

In [ ]:
tick_to_drop = np.concatenate((tick_5letters, tick_dep_rec_no5, adrs_tick_ext, other_adr, other_curr))

In [ ]:
tick_to_drop

In [ ]:
adj_banks_us_df = banks_us_df.copy()
adj_banks_us_df = adj_banks_us_df.set_index('symbol')

In [ ]:
adj_banks_us_df = adj_banks_us_df.drop(index=tick_to_drop)

In [ ]:
adj_banks_us_df[adj_banks_us_df['exchangeShortName'] == 'NASDAQ']

In [ ]:
adj_banks_us_df.reset_index(inplace=True)

In [ ]:
company_list = adj_banks_us_df['symbol'].values 

In [ ]:
nasdaq_list = adj_banks_us_df[adj_banks_us_df['exchangeShortName'] == 'NASDAQ']['symbol'].values

In [ ]:
nyse_list = adj_banks_us_df[adj_banks_us_df['exchangeShortName'] == 'NYSE']['symbol'].values

In [ ]:
amex_list = adj_banks_us_df[adj_banks_us_df['exchangeShortName'] == 'AMEX']['symbol'].values

In [ ]:
#pd.DataFrame(company_list).to_csv('company_list_370.csv')

In [ ]:
len(company_list)

In [ ]:
company_list_list = company_list.tolist()

In [ ]:
company_list_list

##### Financial statements SEC zip

In [ ]:
# download zip files from url
def download_url(url, save_path, chunk_size=128):
    r = requests.get(url, stream=True)
    with open(save_path, 'wb') as fd:
        for chunk in r.iter_content(chunk_size=chunk_size):
            fd.write(chunk)

In [ ]:
# SEC financial statements for each bank 

for sym in company_list:
    gen_url = "https://financialmodelingprep.com/api/v3/financial-statements/{0}?datatype=zip&apikey={1}"
    url = gen_url.format(sym, fmp_apikey)
    download_url(url, 'sec financials\{}.zip'.format(sym))

### Financial statements

In [ ]:
# last 20qtr data for each bank
d_income_statement_quarter = {}

for sym in company_list:
    gen_url = "https://financialmodelingprep.com/api/v3/income-statement/{0}?period=quarter&limit=20&apikey={1}"
    url = gen_url.format(sym, fmp_apikey)
    d_income_statement_quarter[sym] = pd.DataFrame(get_jsonparsed_data(url))

In [ ]:
d_balance_sheet_quarter = {}

for sym in company_list:
    gen_url = "https://financialmodelingprep.com/api/v3/balance-sheet-statement/{0}?period=quarter&limit=20&apikey={1}"
    url = gen_url.format(sym, fmp_apikey)
    d_balance_sheet_quarter[sym] = pd.DataFrame(get_jsonparsed_data(url))

In [ ]:
d_cash_flow_statement_quarter = {}

for sym in company_list:
    gen_url = "https://financialmodelingprep.com/api/v3/cash-flow-statement/{0}?period=quarter&limit=20&apikey={1}"
    url = gen_url.format(sym, fmp_apikey)
    d_cash_flow_statement_quarter[sym] = pd.DataFrame(get_jsonparsed_data(url))

In [ ]:
income_statement_quarter_df = pd.concat(d_income_statement_quarter)

In [ ]:
balance_sheet_quarter_df = pd.concat(d_balance_sheet_quarter)

In [ ]:
cash_flow_statement_quarter_df = pd.concat(d_cash_flow_statement_quarter)

##### Store data 
(current directory)

In [ ]:
#banks_income_statement_last20qtr = income_statement_quarter_df.to_csv('banks_income_statement_last20qtr.csv')

In [ ]:
#banks_balance_sheet_last20qtr = balance_sheet_quarter_df.to_csv('banks_balance_sheet_last20qtr.csv')

In [ ]:
#banks_cash_flow_statement_last20qtr = cash_flow_statement_quarter_df.to_csv('banks_cash_flow_statement_last20qtr.csv')

### Historical market prices

##### Get market prices from yahoo finance 

In [ ]:
prices_yf = yf.download(company_list_list, start="2013-01-01", end="2020-10-30")

In [ ]:
spy_price_yf = yf.download('SPY', start="2013-01-01", end="2020-10-30")

In [ ]:
adj_close_prices_df = prices_yf['Adj Close']

In [ ]:
spy_adj_close_df = spy_price_yf['Adj Close']

In [ ]:
# store data
#adj_close_prices_df.to_csv('adj_close_prices.csv')

In [ ]:
#spy_adj_close_df.to_csv('spy_adj_close.csv')

 (FMP api - historical prices)

In [ ]:
d_stock_prices = {}
    
for sym in company_list:
    gen_url = "https://financialmodelingprep.com/api/v3/historical-price-full/{0}?limit=1512&serietype=line&apikey={1}"
    url = gen_url.format(sym, fmp_apikey)
    d_stock_prices[sym] = pd.DataFrame(get_jsonparsed_data(url))

In [ ]:
close_price_temp_df = pd.concat(d_stock_prices)

In [ ]:
close_price_df = pd.concat([close_price_temp_df.drop(['historical'], axis=1), close_price_temp_df['historical'].apply(pd.Series)], axis=1)

In [ ]:
import pandas_market_calendars as mcal

nyse = mcal.get_calendar('NYSE')
early = nyse.schedule(start_date='2014-01-02', end_date='2020-10-30')
trading_days = mcal.date_range(early, frequency='1D')
standard_date = trading_days.strftime("%Y-%m-%d")
market_prices_df = market_prices_df[market_prices_df['date'] > '2014-01-01']
symbols = market_prices_df['symbol'].values
symbols = list(dict.fromkeys(symbols))
tradDays_df = pd.DataFrame(stand_date, columns=['date'])
tradDays_df = tradDays_df.sort_values('date', ascending=False)

In [ ]:
new_prices_df = dict()
for sym in symbols:
    df_sym = market_prices_df.groupby(level=0).get_group(sym)
    new_prices_df[sym] = pd.merge(tradDays_df, pd.DataFrame(df_sym), how='left')

In [ ]:
prices_df = pd.concat(new_prices_df)
dfs = prices_df.drop(['symbol'], axis=1) 

In [ ]:
df_list = list()
for sym in symbols:
    df_list.append(dfs.groupby(level=0).get_group(sym))

In [ ]:
df_start = pd.DataFrame()
for i in range(len(symbols)):
    df_temp = pd.DataFrame(df_list[i]['close'].values)
    prices_df_adj = pd.concat([df_start, df_temp], axis=1)

In [ ]:
prices_df_adj.columns = symbols
prices_df_adj.insert(0, 'date', tradDays_df.date.values)
index_date = pd.DatetimeIndex(prices_df_adj['date'])
prices_df_adj = prices_df_adj.set_index(index_date)

### Full financial statements as reported

In [ ]:
d_financial_statement_full_quarter = {}

for sym in company_list:
    gen_url = "https://financialmodelingprep.com/api/v3/financial-statement-full-as-reported/{0}?period=quarter&apikey={1}"
    url = gen_url.format(sym, fmp_apikey)
    d_financial_statement_full_quarter[sym] = pd.DataFrame(get_jsonparsed_data(url))

In [ ]:
financial_statement_full_df = pd.concat(d_financial_statement_full_quarter)

In [ ]:
#pd.set_option('display.max_columns', len(financial_statement_full_df.columns))
#pd.set_option('display.max_rows', len(financial_statement_full_df.columns))

In [ ]:
date = financial_statement_full_df['date']
deposits = financial_statement_full_df['deposits']
assets = financial_statement_full_df['assets']
noninterestexpense = financial_statement_full_df['noninterestexpense']
interestexpense = financial_statement_full_df['interestexpense']
noninterestincome = financial_statement_full_df['noninterestincome']
stockholdersequity = financial_statement_full_df['stockholdersequity']

In [ ]:
deposits_df = pd.concat([date, deposits,
                        assets, noninterestexpense,
                        interestexpense, noninterestincome,
                        stockholdersequity], axis=1)

In [ ]:

#deposits_df.to_csv('deposits.csv')

### Market cap

In [ ]:
d_market_cap = {}

for sym in company_list:
    gen_url = "https://financialmodelingprep.com/api/v3/historical-market-capitalization/{0}?limit=1512&apikey={1}"
    url = gen_url.format(sym, fmp_apikey)
    d_market_cap[sym] = pd.DataFrame(get_jsonparsed_data(url))

In [ ]:
market_cap = pd.concat(d_market_cap)

### Historical stock split

In [ ]:
d_stock_split = {}

for sym in company_list:
    gen_url = "https://financialmodelingprep.com/api/v3/historical-price-full/stock_split/{0}?apikey={1}"
    url = gen_url.format(sym, fmp_apikey)
    d_stock_split[sym] = pd.DataFrame(get_jsonparsed_data(url))

In [ ]:
stock_split = pd.concat(d_stock_split)

In [ ]:
stock_split_df = pd.concat([stock_split.drop(['historical'], axis=1), stock_split['historical'].apply(pd.Series)], axis=1)

In [ ]:
stock_split_df[stock_split_df['date'] > '2016-04-01']

### Profile

In [ ]:
d_profile = {}

for sym in company_list:
    gen_url = "https://financialmodelingprep.com/api/v3/profile/{0}?apikey={1}"
    url = gen_url.format(sym, fmp_apikey)
    d_profile[sym] = pd.DataFrame(get_jsonparsed_data(url))

In [ ]:
profile_df = pd.concat(d_profile)

In [ ]:
pd.set_option('display.max_columns', len(profile_df.columns))
pd.set_option('display.max_rows', len(profile_df))

In [ ]:
profile_df

In [ ]:
#profile_df.to_csv('banks_profile.csv')

In [ ]:
profile_df[profile_df['currency'] != 'USD']

In [ ]:
profile_df[profile_df['country'] != 'US']

In [ ]:
len(profile_df[profile_df['ipoDate'] > '2016-01-01'])